# `CLOUD.jl` - 1D linear advection-diffusion example

In [1]:
using CLOUD, OrdinaryDiffEq, Plots, Plots.PlotMeasures

## Define the physical problem

In [2]:
a = 1.0  # wave speed
b = 5.0e-2 # diffusion coefficient
L = 1.0  # domain length
T = 1.0  # end time for one period

initial_data = InitialDataSine(1.0,2π)
conservation_law = LinearAdvectionDiffusionEquation(a,b)
exact_solution = ExactSolution(conservation_law,initial_data);

## Set discretization parameters

In [3]:
M = 4
p = 5
form = WeakConservationForm()
strategy = PhysicalOperator()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.05*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*100));

## Set up a `NodalTensor` scheme of degree `p` with `M` elements

In [4]:
reference_approximation = ReferenceApproximation(
    NodalTensor(p), Line())

mesh = uniform_periodic_mesh(
    reference_approximation.reference_element, (0.0,L), M)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), strategy,
     "results/advection_diffusion_1d/", overwrite=true, clear=true);

## Run the solver and print timer outputs

In [5]:
ode_problem = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)
CLOUD_reset_timer!()
sol = solve(ode_problem, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, (0.0,T), write_interval))
CLOUD_print_timer()

tspan = (0.0, 1.0)
 ────────────────────────────────────────────────────────────────────────────────
            Thread 1                    Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            21.3s /  19.5%           2.29GiB /  18.0%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 local residual         18.1k    1.82s   43.7%   100μs    223MiB   53.0%  12.6KiB
   eval inv num flux    18.1k    844ms   20.3%  46.7μs   45.9MiB   10.9%  2.60KiB
   eval flux            18.1k    237ms    5.7%  13.1μs   22.7MiB    5.4%  1.29KiB
   eval visc num flux   18.1k    235ms    5.7%  13.0μs   35.8MiB    8.5%  2.03KiB
   apply operators      18.1k    175ms    4.2%  9.70μs   33.7MiB    8.0%  1.91KiB
     volume terms       18.1k   34.4ms    0.8%  1.91μs   8.82MiB    2.1%     5

## Calculate the integral L<sup>2</sup> error norm

In [6]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
println("L2 error:")
println(analyze(error_analysis, last(sol.u), exact_solution, T)...)

L2 error:
1.4856115459833738e-6


## Analyze conservation and energy dissipation

In [ ]:
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
p1 = plot(analyze(conservation_analysis, load_time_steps(results_path)), ylabel="Conservation Functional");
p2 = plot(analyze(energy_analysis, load_time_steps(results_path)), ylabel="Energy Functional", legend=:bottomright);
plot(p1,p2, size=(850,300),margin=5mm)

## Plot the exact and numerical solutions

In [ ]:
plot(spatial_discretization, last(sol.u), 
    exact_solution=exact_solution, time=T, ylims=[-1.0,1.0])

## Animate the numerical solution

In [ ]:
using Printf
anim = @animate for i ∈ eachindex(sol.u)
    plot(spatial_discretization, sol.u[i], 
        ylims=[-1.0,1.0], ylabel="\$U^h(x,t)\$",
        label=string("t = ", @sprintf "%.2f" sol.t[i]))
end
gif(anim, "advection_diffusion_solution.gif", fps = 10)